In [ ]:
import sqlite3
import pandas as pd
import numpy as np

#Selecionar arquivo CSV na janela do Windows que servirá como base de dados
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
caminhoArquivoDados = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])

In [ ]:
#### le arquivo CSV e cria DataFrame "df" ####

df = pd.read_csv(caminhoArquivoDados)

# Exclui linhas com valores faltantes
df.dropna(inplace=True)

# Exclui linhas com valores inválidos
condicaoNotasInvalidas = ((df['math score'] > 100) | (df['math score'] < 0) | 
                         (df['reading score'] > 100) | (df['reading score'] < 0) | 
                         (df['writing score'] > 100) | (df['writing score'] < 0))

df.drop(df.loc[condicaoNotasInvalidas].index, inplace=True)

In [ ]:
### Cria novo DataFrame "dfFato", cópia de "df" mas apenas com os campos de notas ###

dfFato = df[['math score', 'reading score', 'writing score']].copy(deep=True)

#Inserir coluna ID no dataframe fato
dfFato.insert(0, 'ID', dfFato.index)